In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('PySp_6').getOrCreate()

In [4]:
spark

In [5]:
df_pysp=spark.read.option('header','true').csv(r"C:\Users\91798\Desktop\Pyspark\test1.csv",header=True,inferSchema=True)

In [6]:
df_pysp.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
df_pysp.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
df_pysp.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
# [Age,Experience]----> new feature--->independent feature # [Age,Experience is a new feature for all]

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [10]:
output=featureassembler.transform(df_pysp)

In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data=output.select("Independent Features","Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [13]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [14]:
regressor.coefficients

DenseVector([-176.3341, 1320.1856])

In [15]:
regressor.intercept

19812.064965197405

In [18]:
y_pred=regressor.evaluate(test_data)

In [21]:
y_pred.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17429.234338747105|
|         [31.0,10.0]| 30000| 27547.56380510443|
+--------------------+------+------------------+



In [22]:
y_pred.meanAbsoluteError,y_pred.meanSquaredError

(2440.8352668213374, 5957811.381290973)